In [4]:
%pip -q install mlflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mlflow

mlflow.set_tracking_uri("http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)

🏃 View run powerful-auk-272 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/0/runs/d21124a57bd1407ba26928aeea035830
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/0
